In [2]:
import requests
import csv
from bs4 import BeautifulSoup

In [2]:
def getTable(link):
    data = []
    r = requests.get(link)
    
    doc = BeautifulSoup(r.text, "html.parser")
    table = doc.find("table")
    table_body = table.find("tbody")
    rows = table_body.find_all("tr")
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
#     for d in data:
#         for dd in d:
#             dd = dd.replace(u'\xad', u'-',2)
    return data

In [6]:
data = getTable("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")
data

[['Baden-Württem\xadberg', '503.115', '258', '1.068', '9,6', '10.354'],
 ['Bayern', '650.901', '333', '1.454', '11,1', '15.309'],
 ['Berlin', '181.593', '251', '800', '21,8', '3.576'],
 ['Branden\xadburg', '108.971', '0', '121', '4,8', '3.815'],
 ['Bremen', '27.662', '23', '95', '13,9', '492'],
 ['Hamburg', '78.047', '64', '276', '14,9', '1.606'],
 ['Hessen', '292.951', '228', '946', '15,0', '7.569'],
 ['Meck\xadlenburg-\nVor\xadpommern', '44.253', '6', '49', '3,0', '1.165'],
 ['Nieder\xadsachsen', '262.887', '232', '1.012', '12,7', '5.805'],
 ['Nord\xadrhein-West\xadfalen', '821.135', '501', '2.480', '13,8', '17.254'],
 ['Rhein\xadland-Pfalz', '156.305', '136', '494', '12,1', '3.895'],
 ['Saarland', '41.759', '44', '133', '13,5', '1.029'],
 ['Sachsen', '286.233', '24', '118', '2,9', '10.121'],
 ['Sachsen-Anhalt', '99.368', '13', '65', '3,0', '3.436'],
 ['Schles\xadwig-Holstein', '64.436', '59', '289', '10,0', '1.628'],
 ['Thüringen', '128.997', '31', '102', '4,8', '4.362'],
 ['Gesamt'

In [22]:
import requests
import mysql.connector
import time

from bs4 import BeautifulSoup
from datetime import datetime


mydb = mysql.connector.connect(
  host="localhost",
  user="test",
  password="uaLAygor",
  database="gr8covid"
)

mycursor = mydb.cursor()

link = "https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html"
r = requests.get(link)
doc = BeautifulSoup(r.text, "html.parser")

table = doc.find("table")
table_body = table.find("tbody")
rows = table_body.find_all("tr")

bundeslaender = ["baden-wuerttemberg","bayern","berlin","brandenburg","bremen","hamburg", "hessen","mecklenburg-vorpommern","niedersachsen","nordrhein-westfalen","rheinland-pfalz","saarland","sachsen","sachsen-anhalt","schleswig-holstein","thueringen"]
mydb.rollback()

for row,b in zip(rows, bundeslaender):
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data = [ele for ele in cols if ele]
    data = data[1:]

    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    timee = doc.find_all("p")[12]
    t_time = str(timee)[19:35].replace(",", "")
    zeitStand = datetime.strptime(t_time,'%d.%m.%Y %H:%M').strftime('%Y-%m-%d %H:%M:%S')
    zeitStand = doc.find_all("p")[12]
    zeitStand = str(zeitStand)[19:35].replace(",", "")
    zeitStand = datetime.strptime(t_time,'%d.%m.%Y %H:%M').strftime('%Y-%m-%d %H:%M:%S')

    anzahl = int(str(data[0]).replace(".", ""))
    diff = int(str(data[1]).replace(".", ""))
    faelle7 = int(str(data[2]).replace(".", ""))
    inzidenz7 = float(data[3].replace(",", "."))
    todesfaelle = int(str(data[4]).replace(".", ""))

    rawZeitStand = str(doc.find_all("p")[12])
    rawAnzahl = str(data[0])
    rawDiff = str(data[1])
    rawFaelle7 = str(data[2])
    rawInzidenz7 = str(data[3])
    rawTodesfaelle = str(data[4])

    html_raw = str(doc)

    mySql_insert_query = "INSERT INTO covid19_cases_rki  (zeit_abgerufen ,zeit_datenstand,bundesland ,anzahl ,differenz_zum_vortag ,faelle_letzte_7_tage ,_7_tage_inzidenz , todesfaelle ,zeit_datenstand_raw ,anzahl_raw ,differenz_zum_vortag_raw ,faelle_letzte_7_tage_raw ,_7_tage_inzidenz_raw ,todesfaelle_raw ,html_raw ) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    record = (timestamp, zeitStand, b, anzahl, diff, faelle7, inzidenz7, todesfaelle, rawZeitStand[:20], rawAnzahl, rawDiff, rawFaelle7, rawInzidenz7, rawTodesfaelle, html_raw)
    mycursor.execute(mySql_insert_query, record)

mydb.commit()
mycursor.close()
mydb.close()  

In [18]:
mydb.commit()

In [37]:
import time
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
t_time = str(timee)[19:35].replace(",", "")
print(t_time)
time_stand = datetime.strptime(t_time,'%d.%m.%Y %H:%M').strftime('%Y-%m-%d %H:%M:%S')
print(time)
print(timestamp)
print(time_stand)

23.7.2021 00:00
<module 'time' (built-in)>
2021-07-24 17:23:14
2021-07-23 00:00:00


In [75]:
import pandas as pd
import numpy as np
import datetime
arr = np.array(data)
head = ["Bundesland", "Anzahl","Differenz zum Vortag","Fälle in den letzten 7 Tagen","7-Tage-Inzidenz","Todesfälle"]
file = pd.DataFrame(data=arr,columns=head)
date = str(datetime.date.today())
file.to_csv("./GermanyCovidCases_"+date+".csv")